# Huấn luyện các mô hình Machine Learning cổ điển (Baseline)

**Mục tiêu**: Sử dụng bộ đặc trưng Scattering Transform đã được trích xuất để huấn luyện và so sánh hiệu suất của các mô hình ML cổ điển, bao gồm Random Forest, XGBoost, và SVM.

**Quy trình**: 
1. **Tải dữ liệu**: Tải bộ đặc trưng Scattering đã được cân bằng.
2. **Phân chia Train/Test (80/20)**: Chia dữ liệu thành một tập training lớn (`_base`) và một tập test cuối cùng (`_final`) để đánh giá khách quan.
3. **Cross-Validation (3-fold)**: Thực hiện 3-fold cross-validation trên tập `_base` để so sánh và lựa chọn mô hình tốt nhất.
4. **Đánh giá cuối cùng**: Huấn luyện mô hình tốt nhất trên toàn bộ tập `_base` và đánh giá hiệu suất trên tập `_final`.

In [ ]:
# Cell 1: Imports

# Thư viện xử lý dữ liệu
import numpy as np
import os
import time
import pandas as pd

# Các mô hình từ scikit-learn và XGBoost
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb

# Các công cụ đánh giá và Cross-Validation
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Thư viện vẽ biểu đồ
import matplotlib.pyplot as plt
import seaborn as sns

# Cài đặt style cho biểu đồ
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

In [ ]:
# Cell 2: Tải dữ liệu và Tách Train/Test Lần Đầu

PROCESSED_DATA_DIR = '../data/processed/'

# Tải toàn bộ dữ liệu đã cân bằng
print("Đang tải dữ liệu đặc trưng Scattering...")
X_data = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_scattering_balanced.npz'))
X = X_data['features']
y = np.load(os.path.join(PROCESSED_DATA_DIR, 'y_labels_balanced.npy'))

print(f"Đã tải thành công bộ dữ liệu đầy đủ với {X.shape[0]} mẫu.")

# Tách dữ liệu thành tập training lớn (base) và tập test cuối cùng (final)
# Sử dụng stratify=y để đảm bảo tỷ lệ các lớp được giữ nguyên trong cả hai tập
X_train_base, X_test_final, y_train_base, y_test_final = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print("Đã chia dữ liệu thành công:")
print(f"- Tập Training Base: {X_train_base.shape[0]} mẫu ({len(X_train_base)/len(X)*100:.1f}%)")
print(f"- Tập Test Final:   {X_test_final.shape[0]} mẫu ({len(X_test_final)/len(X)*100:.1f}%)")

In [ ]:
# Cell 3: Thực hiện 3-fold Cross-Validation trên tập Training Base

# Định nghĩa các mô hình baseline để so sánh
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "SVM (Linear)": SVC(kernel='linear', probability=True, random_state=42) 
}

# Thiết lập chiến lược 3-fold Stratified Cross-Validation
cv_strategy = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

results = []

print("Bắt đầu quá trình Cross-Validation trên tập Training Base...")

for model_name, model in models.items():
    print(f"--- Đang đánh giá {model_name}...")
    start_time = time.time()
    
    # Sử dụng cross_val_score để tính accuracy trung bình qua 3 folds
    scores = cross_val_score(model, X_train_base, y_train_base, cv=cv_strategy, scoring='accuracy', n_jobs=-1)
    
    end_time = time.time()
    
    # Lưu kết quả
    results.append({
        'model': model_name,
        'mean_accuracy': scores.mean(),
        'std_accuracy': scores.std(),
        'cv_time': end_time - start_time
    })
    print(f"Hoàn thành trong {end_time - start_time:.2f} giây.")

# Chuyển kết quả thành DataFrame để dễ xem
results_df = pd.DataFrame(results).sort_values(by='mean_accuracy', ascending=False)

print("=== KẾT QUẢ CROSS-VALIDATION ===")
print(results_df)

In [ ]:
# Cell 4: Huấn luyện và Đánh giá Cuối Cùng trên Tập Test Final

# Chọn mô hình tốt nhất dựa trên mean_accuracy từ CV
best_model_name = results_df.iloc[0]['model']
best_model = models[best_model_name]

print(f"Mô hình tốt nhất được chọn: {best_model_name}")

# Huấn luyện lại mô hình tốt nhất trên toàn bộ tập Training Base
print(f"Đang huấn luyện lại mô hình {best_model_name} trên toàn bộ {X_train_base.shape[0]} mẫu training...")
start_train_time = time.time()
best_model.fit(X_train_base, y_train_base)
end_train_time = time.time()
print(f"Hoàn thành huấn luyện trong {end_train_time - start_train_time:.2f} giây.")

# Đánh giá hiệu suất cuối cùng trên tập Test Final (dữ liệu chưa từng thấy)
print(f'--- ĐÁNH GIÁ CUỐI CÙNG TRÊN {X_test_final.shape[0]} MẪU TEST FINAL ---')
y_pred_final = best_model.predict(X_test_final)

# Tính accuracy
final_accuracy = accuracy_score(y_test_final, y_pred_final)
print(f'\nFinal Test Accuracy: {final_accuracy:.4f}')

# In báo cáo chi tiết
print(classification_report(y_test_final, y_pred_final, target_names=['Absent', 'Present']))

# Vẽ Confusion Matrix
cm = confusion_matrix(y_test_final, y_pred_final)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Absent', 'Present'], yticklabels=['Absent', 'Present'])
plt.title(f'Confusion Matrix - {best_model_name} on Final Test Set')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Lưu mô hình tốt nhất
MODEL_DIR = '../results/models/'
os.makedirs(MODEL_DIR, exist_ok=True)

# Tạo tên file dựa trên tên mô hình và accuracy
model_filename = f"{best_model_name.lower().replace(' ', '_')}_best.pkl"
model_path = os.path.join(MODEL_DIR, model_filename)

print(f"\n{'='*60}")
print(f"Đang lưu mô hình tốt nhất...")
joblib.dump(best_model, model_path)
print(f"✅ Đã lưu mô hình: {model_path}")
print(f"   - Model: {best_model_name}")
print(f"   - Test Accuracy: {final_accuracy:.4f}")
print(f"{'='*60}")

In [ ]:
# Cell 5: Lưu tất cả các mô hình đã train

print("="*60)
print("LƯU TẤT CẢ CÁC MÔ HÌNH")
print("="*60)

# Train và lưu tất cả các mô hình
for model_name, model in models.items():
    print(f"\n--- Đang train và lưu {model_name}...")
    
    # Train trên toàn bộ training base
    start_time = time.time()
    model.fit(X_train_base, y_train_base)
    train_time = end_time - start_time
    
    # Đánh giá trên test final
    y_pred = model.predict(X_test_final)
    test_accuracy = accuracy_score(y_test_final, y_pred)
    
    # Tạo tên file
    model_filename = f"{model_name.lower().replace(' ', '_').replace('(', '').replace(')', '')}.pkl"
    model_path = os.path.join(MODEL_DIR, model_filename)
    
    # Lưu mô hình
    joblib.dump(model, model_path)
    
    print(f"✅ Đã lưu: {model_filename}")
    print(f"   Test Accuracy: {test_accuracy:.4f}")

print("\n" + "="*60)
print("HOÀN THÀNH! Tất cả mô hình đã được lưu tại:")
print(f"{os.path.abspath(MODEL_DIR)}")
print("="*60)

In [ ]:
# Cell 6: Hướng dẫn Load lại mô hình đã lưu

print("="*60)
print("HƯỚNG DẪN SỬ DỤNG MÔ HÌNH ĐÃ LƯU")
print("="*60)

# Ví dụ: Load mô hình tốt nhất
print("\n1. Load mô hình tốt nhất:")
print(f"   loaded_model = joblib.load('{model_path}')")

# Ví dụ: Dự đoán với mô hình đã load
print("\n2. Sử dụng mô hình để dự đoán:")
print("   predictions = loaded_model.predict(X_new)")
print("   probabilities = loaded_model.predict_proba(X_new)")

# Demo thực tế
print("\n3. Demo - Load và test mô hình:")
loaded_model = joblib.load(model_path)

# Test trên 5 mẫu đầu tiên
sample_predictions = loaded_model.predict(X_test_final[:5])
sample_true_labels = y_test_final[:5]

print(f"\n   Dự đoán trên 5 mẫu test:")
for i in range(5):
    label_name = 'Present' if sample_predictions[i] == 1 else 'Absent'
    true_name = 'Present' if sample_true_labels[i] == 1 else 'Absent'
    match = '✓' if sample_predictions[i] == sample_true_labels[i] else '✗'
    print(f"   Sample {i+1}: Predicted={label_name}, True={true_name} {match}")

print("\n" + "="*60)
print("Các file mô hình đã lưu:")
for filename in os.listdir(MODEL_DIR):
    if filename.endswith('.pkl') or filename.endswith('.h5'):
        filepath = os.path.join(MODEL_DIR, filename)
        filesize = os.path.getsize(filepath) / (1024 * 1024)  # MB
        print(f"  - {filename} ({filesize:.2f} MB)")
print("="*60)